In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Да, это Kannada MNIST на Excel

Нейронные сети — удивительное чудо и движующая сила современных информационных технологий. С ними можно решать самые разные задачи. Одна из базовых — распознавание рукописных цифр.

Суть в том, что самообущаяся свёрточная нейросеть должна работать с матрицей яркости пикселей (28×28) с операциями свёртки специальными фильтрами, объединением (пуллингом), исключением (дроп-аутом).
> Но что если сделать шаг назад в инновациях и использовать не машинное обучение, а Microsoft Excel?

Kaggle [предлагает](https://www.kaggle.com/c/Kannada-MNIST/data) нам четыре файла, один из которых, `train.csv`, мы будем использовать для проверки нашей гипотез.

![](https://psv4.userapi.com/c856528/u99539515/docs/d13/7eecc796b1f9/1.png)

## Импорт данных

Создадим в Excel пустую книгу и импортируем файл `test.csv` с помощью получения внешних данных.
* Кодировка: UTF-8. 
* Разделители: запятые. 
* Формат данных: общий.

Вставляем в имеющийся лист.

![Untitled/Untitled%201.png](https://psv4.userapi.com/c856416/u99539515/docs/d9/6b6410215651/Untitled_1.png)

Данных довольно много, более 47 миллионов ячеек, работать с ними долго и неудобно. Скроем некоторые столбцы (оставим по три начальных, средних и конечных), а также оставим меньше видимых строк. Я оставлю первые десять и пару последних, а также добавлю шапку для этих данных.

![Untitled/Untitled%202.png](https://psv4.userapi.com/c856228/u99539515/docs/d10/40212e6c3738/Untitled_2.png)

Колонка — номер пикселя (указан сверху; картинка «вытянута» в одномерный массив). Строка — это номер картинки, или экземпляра.
Большинство данных — нули. Действительно, большая площадь изображения занята... ничем. Она пустая. Чёрная. Нулевая.

![Untitled/header.png](https://psv4.userapi.com/c856220/u99539515/docs/d13/49519081acf6/header.png)

## Модели цифр

Чтобы распознать рукописную цифру, нам нужно сверить её с моделями существующих цифр. Наша модель будет простой. Используем среднее арифметическое для яркости пикселя у одинаковых цифр.

Итого получим 10 строчек для каждой цифры. Используем функцию `СРЗНАЧЕСЛИ` (AVERAGEIF), считаем в первой ячейке, а затем растягиваем на всю матрицу.

> 47 миллионов ячеек это много, поэтому необходимо постоянно ждать и сохраняться.

![Untitled/Untitled%203.png](https://psv4.userapi.com/c856432/u99539515/docs/d6/cded2e2d7c11/Untitled_3.png)

Да, изначально должен получится ноль. Никто не пишет в углу холста.

Проверяем, нет ли проблем с адресацией. Удобно ориентироваться по цветной подстветке ячеек.

![Untitled/Untitled%204.png](https://psv4.userapi.com/c856528/u99539515/docs/d6/5a56018a06cd/Untitled_4.png)

Я потратил много времени при построении различных моделей цифр. Изначально я планировал найти такую функцию контрольных сумм, которая приблизительно, но более менее однозначно описывала цифры. Но более успешным ходом (и самым успешным из простых) оказалась такая модель со средним арифметическим.

In [ ]:
model = pd.read_csv("../input/kannadamnist-avg/model.csv", header=None)

print("Kannada digits models:")
fig = plt.figure()
for i in range(1, 11):
    number = model.iloc[i-1,0:784].values.reshape(28,28)
    ax = fig.add_subplot(2, 5, i)
    plt.imshow(number, cmap=plt.get_cmap('gray'))

## Проверка на соответствие

«Вытащим» условно-случайную величину из нашего набора данных. Например, из строки 123321. Для этого используем простую адресацию `A12321` и растянем её до 784 пикселя.

Это число оказалось восьмёркой, а видимые нам значения нулевые. Но это, конечно, совсем не так. Можно убедиться в этом, показав скрытые столбцы.

![Untitled/Untitled%205.png](https://psv4.userapi.com/c856528/u99539515/docs/d2/b8290fea3f96/Untitled_5.png)

Для проверки этих данных (по факту, картинки) на соответствие модели, можем воспользоваться разными инструментами. Я буду расчитывать квадрат коэффициента корелляции и ковариацию. Наша случайная цифра будет сравниваться с 10 моделями. Наибольшие параметры предсказывают наибольшее совпадение. 

Используем `КВПИРСОН` (RSQ) для вычисления первого параметра. Поместим их напротив наших моделей для удобства восприятия.

![Untitled/Untitled_(1).png](https://psv4.userapi.com/c856232/u99539515/docs/d11/f2d98420a380/Untitled_1.png)

Внимательные читатели сразу уловят суть, но пока расчитаем ещё один параметр — ковариацию. Оба этих параметра отвечают за связь наборов данных. Аргументы обеих функций совпадают:

![Untitled/Untitled%206.png](https://psv4.userapi.com/c856436/u99539515/docs/d18/aba5ad6ba981/Untitled_6.png)

Для наглядности с помощью условного форматирования построим гистограммы, изменим формат и приведём всё в более наглядный вид. Снизу нужно вывести максимумы среди значений.

![Untitled/Untitled%207.png](https://psv4.userapi.com/c856232/u99539515/docs/d16/f1ab1d16ff04/Untitled_7.png)

Чтобы отобразить номер модели, с которой у случайного экземпляра максимальная корреляция и ковариация (argmax), используем функцию `ПОИСКПОЗ` (MATCH). Она сравнивает ячейку максимума со всеми значениями и выводит номер модели, соответствующей максимуму. Фактически, это и есть вывод нашей системы.

Зачем нам два параметра? Аргументы, при которых максимальны корелляция и ковариация, могуть быть разными. Здесь имеет место быть неопределённость вычислений — экземпляр по-разному, но похож на две можели одновременно. 

![Untitled/Untitled%208.png](https://psv4.userapi.com/c856220/u99539515/docs/d11/18caf29d02a8/Untitled_8.png)

Не мудрствуя лукаво, случайно возьмём один из двух выводов с помощью `СЛУЧМЕЖДУ` (RANDBETWEEN) и `ИНДЕКС` (INDEX) и назначим нашему экземпляру. Пока это наш главный вывод, поэтому сделаем его выразительнее. Сделаем индикатор при помощью белого текста и условного форматирования. Фон будет появляться при выполнении формулы `СОВПАД` (EXACT) у главного вывода и первой колонки случайного экземпляра.

![Untitled/Untitled%209.png](https://psv4.userapi.com/c856324/u99539515/docs/d9/a37d1ea65869/Untitled_9.png)

## Более случайные экземпляры

Вместо придуманного числа [12321] используем функцию `СЛУЧМЕЖДУ` (RANDBETWEEN), которая генирует нам номер случайного экземпляра (от 1 до 60000). Ссылаемся на него с помощью `ИНДЕКС` (INDEX). При каждом изменении в ячейках Excel будет генерировать новое случайное число и 25 значений будут высчитываться заново (по 10 значений квадратного коэффициента корреляции и ковариации, 2 их максимума, 2 argmax и 1 случайный выбор).

![Untitled/Untitled%2010.png](https://psv4.userapi.com/c856432/u99539515/docs/d11/693edd65f325/Untitled_10.png)

Чтобы таким образом проверить на соответствие все экземпляры и вычислить общий процент попаданий, нужно дополнительно 25 × 60 000 = 1 500 000 ячеек. Такое решение «в лоб» не подходит из соображений ресурсоемкости и здравого смысла.

Используем макрос, проходим нескольким результатам (вычисление ограничиваем временем) и сохраняем вычисленную цифру напротив экземпляра.

    Sub calc()
        Application.ScreenUpdating = False
        
        Dim AbortTime As Date
        AbortTime = Now + TimeValue("02:00:00")
        
        Dim i As Long, j As Long
        For i = 3 To 60002
            For j = 1 To 785
                Cells(60015, j) = Cells(i, j)
            Next j
            Cells(i, "ADF") = Cells(60006, "ADL")
            If Now > AbortTime Then Exit For
        Next i
        [ADK2].Value = "Done"
        
        Application.ScreenUpdating = True
    End Sub

Вычисляем число попаданий и получаем точность около **83%**.

## Вычисление на тестовых данных

Импортируем файл `test.csv` подобное тому, как сделали это с `train.csv`.

![Untitled/Untitled%2011.png](https://psv4.userapi.com/c856532/u99539515/docs/d13/4ae795ee956d/Untitled_11.png)

Запустим вычислительный макрос для этих полей. После того, как 5000 значений сосчитаны, вставляем их в файл и выкладываем на Kaggle.

In [ ]:
sample_submission = pd.read_csv("../input/output/sample_submission.csv")
sample_submission.to_csv('submission.csv',index=False)